In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pickle
from model import ATT_CNN_GRU_MultiFeature
import os
import numpy as np

# Hyperparameters
batch_size = 32
learning_rate = 0.0005
num_epochs = 200
in_channels = 18  # Number of features in the dataset (updated)
num_classes = 16  # Total number of fault classes

# Custom Dataset to handle pre-segmented data
class PreSegmentedDataset(Dataset):
    def __init__(self, X, y):
        self.data = X
        self.labels = y

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# Load the preprocessed data
with open('processed_data.pkl', 'rb') as f:
    data_dict = pickle.load(f)

# Extract data and labels
X_train = torch.tensor(data_dict['X_train'], dtype=torch.float32)
X_val = torch.tensor(data_dict['X_val'], dtype=torch.float32)
X_test = torch.tensor(data_dict['X_test'], dtype=torch.float32)
y_train = torch.tensor(data_dict['y_train'], dtype=torch.long)  # Integer labels
y_val = torch.tensor(data_dict['y_val'], dtype=torch.long)
y_test = torch.tensor(data_dict['y_test'], dtype=torch.long)

# Create datasets
train_dataset = PreSegmentedDataset(X_train, y_train)
val_dataset = PreSegmentedDataset(X_val, y_val)
test_dataset = PreSegmentedDataset(X_test, y_test)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize model
model = ATT_CNN_GRU_MultiFeature(in_channels, num_classes)
criterion = nn.CrossEntropyLoss()  # Suitable for integer class labels
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Variables to track the best model
best_val_accuracy = 0.0
best_model_path = ""
best_epoch = 0  # To store the epoch when the best model is saved

# Create a directory to save models if it doesn't exist
if not os.path.exists("models"):
    os.makedirs("models")

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Permute the input to the shape [batch_size, in_channels, sequence_length]
        inputs = inputs.permute(0, 2, 1)  # Shape should now be [batch_size, in_channels, seq_len]
        
        # Forward pass
        outputs = model(inputs)
        
        # Calculate loss
        loss = criterion(outputs, labels)  # Assuming labels are integer class indices
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Accumulate training metrics
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    train_accuracy = 100 * correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {train_accuracy:.2f}%')

    # Validation loop
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            # Permute the input to the shape [batch_size, in_channels, sequence_length]
            inputs = inputs.permute(0, 2, 1)
            
            # Forward pass
            outputs = model(inputs)
            
            # Calculate loss
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    
    val_accuracy = 100 * correct / total
    print(f'Validation Loss: {val_loss/len(val_loader):.4f}, Validation Accuracy: {val_accuracy:.2f}%')

    # Save the best model based on validation accuracy
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_epoch = epoch + 1  # Record the best epoch
        best_model_path = f"models/best_model_lr_{learning_rate}_bs_{batch_size}.pth"
        torch.save(model.state_dict(), best_model_path)
        print(f"New best model saved with validation accuracy: {val_accuracy:.2f}% at epoch {best_epoch}")

# Test loop (testing the best model)
model.load_state_dict(torch.load(best_model_path))
model.eval()
correct = 0
total = 0
all_preds = []
all_labels = []
with torch.no_grad():
    for inputs, labels in test_loader:
        # Permute the input to the shape [batch_size, in_channels, sequence_length]
        inputs = inputs.permute(0, 2, 1)
        
        # Forward pass
        outputs = model(inputs)
        
        # Calculate the accuracy
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
        
        # Store original and predicted labels
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_accuracy = 100 * correct / total
print(f'Test Accuracy: {test_accuracy:.2f}%')

# Print final best model path and epoch
print(f"Best model saved at: {best_model_path} with validation accuracy: {best_val_accuracy:.2f}% (epoch {best_epoch})")


Epoch [1/200], Loss: 2.6493, Accuracy: 36.24%
Validation Loss: 2.6134, Validation Accuracy: 60.34%
New best model saved with validation accuracy: 60.34% at epoch 1
Epoch [2/200], Loss: 2.2103, Accuracy: 65.94%
Validation Loss: 2.1843, Validation Accuracy: 75.86%
New best model saved with validation accuracy: 75.86% at epoch 2
Epoch [3/200], Loss: 1.7404, Accuracy: 63.32%
Validation Loss: 1.5673, Validation Accuracy: 65.52%
Epoch [4/200], Loss: 1.2775, Accuracy: 64.19%
Validation Loss: 1.0351, Validation Accuracy: 82.76%
New best model saved with validation accuracy: 82.76% at epoch 4
Epoch [5/200], Loss: 0.9717, Accuracy: 72.05%
Validation Loss: 0.6500, Validation Accuracy: 84.48%
New best model saved with validation accuracy: 84.48% at epoch 5
Epoch [6/200], Loss: 0.7179, Accuracy: 76.86%
Validation Loss: 0.4923, Validation Accuracy: 87.93%
New best model saved with validation accuracy: 87.93% at epoch 6
Epoch [7/200], Loss: 0.5729, Accuracy: 86.03%
Validation Loss: 0.3386, Validation

Epoch [79/200], Loss: 0.2151, Accuracy: 96.07%
Validation Loss: 0.3140, Validation Accuracy: 89.66%
Epoch [80/200], Loss: 0.2381, Accuracy: 90.39%
Validation Loss: 0.2281, Validation Accuracy: 94.83%
Epoch [81/200], Loss: 0.1590, Accuracy: 98.69%
Validation Loss: 0.2587, Validation Accuracy: 94.83%
Epoch [82/200], Loss: 0.0432, Accuracy: 99.56%
Validation Loss: 0.2866, Validation Accuracy: 91.38%
Epoch [83/200], Loss: 0.0994, Accuracy: 96.51%
Validation Loss: 0.2407, Validation Accuracy: 96.55%
Epoch [84/200], Loss: 0.2196, Accuracy: 96.07%
Validation Loss: 0.2242, Validation Accuracy: 94.83%
Epoch [85/200], Loss: 0.0618, Accuracy: 98.69%
Validation Loss: 0.2561, Validation Accuracy: 96.55%
Epoch [86/200], Loss: 0.4467, Accuracy: 98.25%
Validation Loss: 0.2600, Validation Accuracy: 96.55%
Epoch [87/200], Loss: 0.0598, Accuracy: 98.69%
Validation Loss: 0.2386, Validation Accuracy: 96.55%
Epoch [88/200], Loss: 0.1730, Accuracy: 99.13%
Validation Loss: 0.2029, Validation Accuracy: 96.55%


Epoch [161/200], Loss: 0.0192, Accuracy: 100.00%
Validation Loss: 0.2074, Validation Accuracy: 96.55%
Epoch [162/200], Loss: 0.0133, Accuracy: 99.56%
Validation Loss: 0.2349, Validation Accuracy: 96.55%
Epoch [163/200], Loss: 0.0048, Accuracy: 100.00%
Validation Loss: 0.2345, Validation Accuracy: 96.55%
Epoch [164/200], Loss: 0.0712, Accuracy: 99.13%
Validation Loss: 0.1870, Validation Accuracy: 96.55%
Epoch [165/200], Loss: 0.0098, Accuracy: 99.56%
Validation Loss: 0.2259, Validation Accuracy: 96.55%
Epoch [166/200], Loss: 0.0287, Accuracy: 99.56%
Validation Loss: 0.3079, Validation Accuracy: 96.55%
Epoch [167/200], Loss: 0.0067, Accuracy: 100.00%
Validation Loss: 0.3715, Validation Accuracy: 96.55%
Epoch [168/200], Loss: 0.0246, Accuracy: 99.13%
Validation Loss: 0.2931, Validation Accuracy: 96.55%
Epoch [169/200], Loss: 0.0112, Accuracy: 100.00%
Validation Loss: 0.3170, Validation Accuracy: 96.55%
Epoch [170/200], Loss: 0.0091, Accuracy: 100.00%
Validation Loss: 0.2863, Validation Ac